In [1]:
import pandas as pd

1.5 Feature Engineering

In [4]:
df_merged = pd.read_csv(r'C:\Users\Yibabe\Desktop\10acadamyAMIweek-8-9\data\merged_data.csv')

In [6]:
df_merged.head()

,Unnamed: 0,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,class,ip_int,country
0,634,247547,2015-06-28 03:00:34,2015-08-09 03:57:29,47,KIXYSVCHIPQBR,SEO,Safari,F,30,0,16778864,Australia
1,635,220737,2015-01-28 14:21:11,2015-02-11 20:28:28,15,PKYOWQKWGJNJI,SEO,Chrome,F,34,0,16842045,Thailand
2,636,390400,2015-03-19 20:49:09,2015-04-11 23:41:23,44,LVCSXLISZHVUO,Ads,IE,M,29,0,16843656,China
3,637,69592,2015-02-24 06:11:57,2015-05-23 16:40:14,55,UHAUHNXXUADJE,Direct,Chrome,F,30,0,16938732,China
4,638,174987,2015-07-07 12:58:11,2015-11-03 04:04:30,51,XPGPMOHIDRMGE,SEO,Chrome,F,37,0,16971984,Thailand


1.5.1 1 Calculate Transaction Frequency

In [8]:
# Convert the transaction_time column to datetime format if not already
df_merged['purchase_time'] = pd.to_datetime(df_merged['purchase_time'])

# Group by user_id and day, then count the number of transactions per day
df_merged['day'] = df_merged['purchase_time'].dt.date
transaction_frequency = df_merged.groupby(['user_id', 'day']).size().reset_index(name='transaction_count_per_day')




In [9]:
# Merge back with the merged dataset
df_merged = df_merged.merge(transaction_frequency, on=['user_id', 'day'], how='left')

1..5.2 Calculate Transaction Velocity

In [11]:
# Sort by user_id and transaction_time
df_merged = df_merged.sort_values(by=['user_id', 'purchase_time'])

# Calculate the time difference between consecutive transactions for each user
df_merged['time_diff'] = df_merged.groupby('user_id')['purchase_time'].diff().dt.total_seconds()

# Fill missing values in time_diff (first transaction for each user) with a large number or zero
df_merged['time_diff'] = df_merged['time_diff'].fillna(0)


 Time-Based Features

In [12]:
# Extract hour of the day
df_merged['hour_of_day'] = df_merged['purchase_time'].dt.hour


 Day of the Week

In [13]:
# Extract day of the week
df_merged['day_of_week'] = df_merged['purchase_time'].dt.dayofweek


In [14]:
df_merged.head()

,Unnamed: 0,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,class,ip_int,country,day,transaction_count_per_day,time_diff,hour_of_day,day_of_week
30049,31545,2,2015-01-11 03:47:13,2015-02-21 10:03:37,54,FGBQNDNBETFJJ,SEO,Chrome,F,25,0,880217484,United States,2015-02-21,1,0.0,10,5
95244,97542,4,2015-06-02 16:40:57,2015-09-26 21:32:16,41,MKFUIVOHLJBYN,Direct,Safari,F,38,0,2785906106,Switzerland,2015-09-26,1,0.0,21,5
11606,12873,8,2015-05-28 07:53:06,2015-08-13 11:53:07,47,SCQGQALXBUQZJ,SEO,Chrome,M,25,0,356056736,United States,2015-08-13,1,0.0,11,3
101959,104500,12,2015-01-10 06:25:12,2015-03-04 20:56:37,35,MSNWCFEHKTIOY,Ads,Safari,M,19,0,2985180352,Mexico,2015-03-04,1,0.0,20,2
19600,21011,16,2015-02-03 13:48:23,2015-03-12 12:46:23,9,FROZWSSWOHZBE,Direct,IE,M,32,0,578312545,United States,2015-03-12,1,0.0,12,3


In [15]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the Min-Max Scaler
scaler = MinMaxScaler()

# List of columns to scale
columns_to_scale = ['purchase_value', 'age', 'ip_int', 'transaction_count_per_day', 
                    'time_diff', 'hour_of_day', 'day_of_week']

# Apply normalization
df_merged[columns_to_scale] = scaler.fit_transform(df_merged[columns_to_scale])


In [16]:
df1_merged = df_merged.copy()

In [18]:
df1_merged.head()

,Unnamed: 0,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,class,ip_int,country,day,transaction_count_per_day,time_diff,hour_of_day,day_of_week
30049,31545,2,2015-01-11 03:47:13,2015-02-21 10:03:37,0.310345,FGBQNDNBETFJJ,SEO,Chrome,F,0.120690,0,0.230787,United States,2015-02-21,0.0,0.0,0.434783,0.833333
95244,97542,4,2015-06-02 16:40:57,2015-09-26 21:32:16,0.220690,MKFUIVOHLJBYN,Direct,Safari,F,0.344828,0,0.740156,Switzerland,2015-09-26,0.0,0.0,0.913043,0.833333
11606,12873,8,2015-05-28 07:53:06,2015-08-13 11:53:07,0.262069,SCQGQALXBUQZJ,SEO,Chrome,M,0.120690,0,0.090685,United States,2015-08-13,0.0,0.0,0.478261,0.500000
101959,104500,12,2015-01-10 06:25:12,2015-03-04 20:56:37,0.179310,MSNWCFEHKTIOY,Ads,Safari,M,0.017241,0,0.793420,Mexico,2015-03-04,0.0,0.0,0.869565,0.333333
19600,21011,16,2015-02-03 13:48:23,2015-03-12 12:46:23,0.000000,FROZWSSWOHZBE,Direct,IE,M,0.241379,0,0.150092,United States,2015-03-12,0.0,0.0,0.521739,0.500000


next step is scalled the numerical columns and encode the catagorical coumns

In [19]:
# List of categorical columns to check
categorical_columns = ['source', 'browser', 'sex', 'country']

# Display the number of unique values for each categorical column
for col in categorical_columns:
    unique_values = df_merged[col].nunique()
    print(f"{col}: {unique_values} unique values")


source: 3 unique values
browser: 5 unique values
sex: 2 unique values
country: 181 unique values


use hot encoding and label encoding based on the number of unique value of the columns

In [20]:
from sklearn.preprocessing import LabelEncoder

# One-Hot Encode 'source', 'browser', and 'sex'
df_encoded = pd.get_dummies(df_merged, columns=['source', 'browser', 'sex'], drop_first=True)

# Label Encode 'country'
label_encoder = LabelEncoder()
df_encoded['country'] = label_encoder.fit_transform(df_encoded['country'])

In [21]:
# Display the first few rows of the encoded dataframe
df_encoded.head()

,Unnamed: 0,user_id,signup_time,purchase_time,purchase_value,device_id,age,class,ip_int,country,...,time_diff,hour_of_day,day_of_week,source_Direct,source_SEO,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M
30049,31545,2,2015-01-11 03:47:13,2015-02-21 10:03:37,0.310345,FGBQNDNBETFJJ,0.120690,0,0.230787,171,...,0.0,0.434783,0.833333,False,True,False,False,False,False,False
95244,97542,4,2015-06-02 16:40:57,2015-09-26 21:32:16,0.220690,MKFUIVOHLJBYN,0.344828,0,0.740156,157,...,0.0,0.913043,0.833333,True,False,False,False,False,True,False
11606,12873,8,2015-05-28 07:53:06,2015-08-13 11:53:07,0.262069,SCQGQALXBUQZJ,0.120690,0,0.090685,171,...,0.0,0.478261,0.500000,False,True,False,False,False,False,True
101959,104500,12,2015-01-10 06:25:12,2015-03-04 20:56:37,0.179310,MSNWCFEHKTIOY,0.017241,0,0.793420,107,...,0.0,0.869565,0.333333,False,False,False,False,False,True,True
19600,21011,16,2015-02-03 13:48:23,2015-03-12 12:46:23,0.000000,FROZWSSWOHZBE,0.241379,0,0.150092,171,...,0.0,0.521739,0.500000,True,False,False,True,False,False,True


In [22]:
# Convert boolean columns to integer (0 and 1)
df_encoded[['source_Direct', 'source_SEO', 'browser_FireFox', 'browser_IE', 'browser_Opera', 'browser_Safari', 'sex_M']] = \
    df_encoded[['source_Direct', 'source_SEO', 'browser_FireFox', 'browser_IE', 'browser_Opera', 'browser_Safari', 'sex_M']].astype(int)



In [23]:

# Display the first few rows to verify the changes
df_encoded.head()

,Unnamed: 0,user_id,signup_time,purchase_time,purchase_value,device_id,age,class,ip_int,country,...,time_diff,hour_of_day,day_of_week,source_Direct,source_SEO,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M
30049,31545,2,2015-01-11 03:47:13,2015-02-21 10:03:37,0.310345,FGBQNDNBETFJJ,0.120690,0,0.230787,171,...,0.0,0.434783,0.833333,0,1,0,0,0,0,0
95244,97542,4,2015-06-02 16:40:57,2015-09-26 21:32:16,0.220690,MKFUIVOHLJBYN,0.344828,0,0.740156,157,...,0.0,0.913043,0.833333,1,0,0,0,0,1,0
11606,12873,8,2015-05-28 07:53:06,2015-08-13 11:53:07,0.262069,SCQGQALXBUQZJ,0.120690,0,0.090685,171,...,0.0,0.478261,0.500000,0,1,0,0,0,0,1
101959,104500,12,2015-01-10 06:25:12,2015-03-04 20:56:37,0.179310,MSNWCFEHKTIOY,0.017241,0,0.793420,107,...,0.0,0.869565,0.333333,0,0,0,0,0,1,1
19600,21011,16,2015-02-03 13:48:23,2015-03-12 12:46:23,0.000000,FROZWSSWOHZBE,0.241379,0,0.150092,171,...,0.0,0.521739,0.500000,1,0,0,1,0,0,1


save the scalled and encoded file to the next analysis

In [25]:
df_encoded.to_csv(r'C:\Users\Yibabe\Desktop\10acadamyAMIweek-8-9\data\scaled_encoded.csv')